# Building a Multilayer Convolutional Network

the previous tutorial is using softmax regression to recognize MNIST digits.

The tutorial for this notebook is here:

[Building a Multilayer Convolutional Network](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html#deep-mnist-for-experts#content)

In [1]:
import tensorflow as tf

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [5]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
layer_conv_1 = conv2d(x_image, W_conv1) + b_conv1

In [9]:
W_conv2 = weight_variable([5, 5, 32, 32])
b_conv2 = bias_variable([32])
layer_conv_2 = conv2d(layer_conv_1, W_conv2) + b_conv2

stage_1_pool = max_pool_2x2(layer_conv_2)

In [10]:
W_conv3 = weight_variable([5, 5, 32, 64])
b_conv3 = bias_variable([64])
layer_conv_3 = conv2d(layer_conv_2, W_conv3) + b_conv3

W_conv4 = weight_variable([5, 5, 64, 64])
b_conv4 = bias_variable([64])
layer_conv_4 = conv2d(layer_conv_3, W_conv4) + b_conv4

stage_2_pool = max_pool_2x2(layer_conv_4)

## Stage 3:

In [11]:
W_conv5 = weight_variable([5, 5, 64, 128])
b_conv5 = bias_variable([128])
layer_conv_5 = conv2d(layer_conv_4, W_conv5) + b_conv5

W_conv6 = weight_variable([5, 5, 128, 128])
b_conv6 = bias_variable([128])
layer_conv_6 = conv2d(layer_conv_5, W_conv6) + b_conv6

stage_3_pool = max_pool_2x2(layer_conv_6)

In [12]:
stage_3_pool.get_shape()

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(128)])

In [13]:
W_fc1 = weight_variable([14 * 14 * 128, 2])
b_fc1 = bias_variable([2])

stage_3_pool_flat = tf.reshape(stage_3_pool, [-1, 14 * 14 * 128])
h_fc1 = tf.nn.relu(tf.matmul(stage_3_pool_flat, W_fc1) + b_fc1)

In [14]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [15]:
W_fc2 = weight_variable([2, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
sess = tf.InteractiveSession()

In [ ]:
%%time
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(40000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.06
step 100, training accuracy 0.12
step 200, training accuracy 0.1
step 300, training accuracy 0.06
step 400, training accuracy 0.08
step 500, training accuracy 0.08
step 600, training accuracy 0.06
step 700, training accuracy 0.14
step 800, training accuracy 0.06
step 900, training accuracy 0.1
step 1000, training accuracy 0.14
step 1100, training accuracy 0.06
step 1200, training accuracy 0.12
step 1300, training accuracy 0.1
step 1400, training accuracy 0.12
step 1500, training accuracy 0.06
step 1600, training accuracy 0.02
step 1700, training accuracy 0.06
step 1800, training accuracy 0.12
step 1900, training accuracy 0.12

In [ ]:
%%time
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images[:250], y_: mnist.test.labels[:250], keep_prob: 1.0}))